In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 1. Chargement du tokenizer et des modèles
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model_general = BertForSequenceClassification.from_pretrained("path_to_general_model")
model_covid = BertForSequenceClassification.from_pretrained("path_to_covid_model")

model_general.eval()
model_covid.eval()

# 2. Chargement du jeu de données COVID
df_test = pd.read_csv("covid_test_dataset.csv")

texts = df_test["text"].tolist()
labels = df_test["label"].tolist()  # 0 ou 1

# 3. Fonction utilitaire pour prédire avec un modèle donné
def predict_with_model(model, text_list):
    predictions = []
    with torch.no_grad():
        for text in text_list:
            encoding = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
            outputs = model(**encoding)
            pred = torch.argmax(outputs.logits, dim=1).item()
            predictions.append(pred)
    return predictions

# 4. Obtenir les prédictions de chaque modèle
preds_general = predict_with_model(model_general, texts)
preds_covid = predict_with_model(model_covid, texts)

# 5. Calcul et affichage des performances
# Accuracy
acc_general = accuracy_score(labels, preds_general)
acc_covid = accuracy_score(labels, preds_covid)

# F1-score (macro ou weighted selon vos besoins)
f1_general = f1_score(labels, preds_general, average="weighted")
f1_covid = f1_score(labels, preds_covid, average="weighted")

print("=== Performances du modèle Général ===")
print(f"Accuracy : {acc_general:.4f}")
print(f"F1-score : {f1_general:.4f}")
print(classification_report(labels, preds_general, digits=4))

print("\n=== Performances du modèle COVID ===")
print(f"Accuracy : {acc_covid:.4f}")
print(f"F1-score : {f1_covid:.4f}")
print(classification_report(labels, preds_covid, digits=4))
